In [1]:
import numpy as np
import pandas as pd

train_set = pd.read_csv('./dataset/train.csv').sample(frac=1.0)#[:1000]
test_set = pd.read_csv('./dataset/test.csv')#[:1]
train_set.head(10)

,时间,小区名,小区房屋出租数量,楼层,总楼层,房屋面积,房屋朝向,居住状态,卧室数量,厅的数量,卫的数量,出租方式,区,位置,地铁线路,地铁站点,距离,装修情况,月租金
34333,1,6556,0.117188,2,0.581818,0.014068,东南,NaN,3,1,2,NaN,8.0,80.0,NaN,NaN,NaN,NaN,5.602716
57240,2,4659,0.050781,1,0.600000,0.013241,东南,NaN,2,2,1,NaN,12.0,15.0,NaN,NaN,NaN,NaN,6.621392
58959,2,1597,0.121094,1,0.454545,0.017875,东南,NaN,3,2,2,NaN,1.0,40.0,NaN,NaN,NaN,NaN,6.281834
163930,3,6221,0.535156,2,0.563636,0.014399,南,NaN,2,2,1,NaN,12.0,52.0,NaN,NaN,NaN,NaN,10.696095
171982,3,5239,0.484375,2,0.545455,0.008937,北,NaN,1,0,1,NaN,1.0,38.0,5.0,27.0,0.228333,NaN,8.998302
55511,2,472,0.257812,1,0.272727,0.012579,南,NaN,2,2,1,NaN,0.0,123.0,NaN,NaN,NaN,NaN,4.244482
10600,1,903,0.050781,1,0.600000,0.010096,东南,NaN,2,2,1,NaN,10.0,141.0,NaN,NaN,NaN,NaN,4.923599
5305,1,3146,0.128906,0,0.327273,0.013737,东,NaN,3,2,1,NaN,9.0,22.0,NaN,NaN,NaN,NaN,4.923599
18314,1,1247,0.054688,1,0.581818,0.013903,西,NaN,2,2,1,NaN,12.0,54.0,NaN,NaN,NaN,NaN,9.677419
191277,3,1910,0.132812,0,0.527273,0.010013,北,NaN,2,1,1,NaN,4.0,51.0,NaN,NaN,NaN,NaN,2.546689


In [2]:
train_set.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 196539 entries, 34333 to 24280
Data columns (total 19 columns):
时间          196539 non-null int64
小区名         196539 non-null int64
小区房屋出租数量    195538 non-null float64
楼层          196539 non-null int64
总楼层         196539 non-null float64
房屋面积        196539 non-null float64
房屋朝向        196539 non-null object
居住状态        20138 non-null float64
卧室数量        196539 non-null int64
厅的数量        196539 non-null int64
卫的数量        196539 non-null int64
出租方式        24230 non-null float64
区           196508 non-null float64
位置          196508 non-null float64
地铁线路        91778 non-null float64
地铁站点        91778 non-null float64
距离          91778 non-null float64
装修情况        18492 non-null float64
月租金         196539 non-null float64
dtypes: float64(12), int64(6), object(1)
memory usage: 30.0+ MB


In [3]:
train_set.corr()['月租金'] #相关系数

时间         -0.014485
小区名         0.007568
小区房屋出租数量    0.011822
楼层          0.009495
总楼层         0.139803
房屋面积        0.440718
居住状态       -0.075957
卧室数量        0.315892
厅的数量        0.221741
卫的数量        0.423756
出租方式        0.247789
区           0.146881
位置          0.006541
地铁线路        0.187766
地铁站点        0.019662
距离         -0.075449
装修情况        0.246597
月租金         1.000000
Name: 月租金, dtype: float64

In [4]:
cols_oh = ['小区名','房屋朝向','区','位置','地铁线路','地铁站点', #其中 区 位置 地铁线路 地铁站点有缺失
          ]
cols_oh_train = cols_oh+['月租金']
cols_oh_test = cols_oh+['id']
cols_fill_0 = ('装修情况')
cols_fill_None = ['居住状态','出租方式','区','位置','地铁线路','地铁站点',] #？？？
cols_fill_mean = ['小区房屋出租数量','距离',
                 ]
# cols_nums = ['时间','小区房屋出租数量','楼层','总楼层','房屋面积','卧室数量','厅的数量','卫的数量','距离','装修情况']

X_train = train_set.drop(['月租金'], axis=1)
y_train = train_set['月租金']
X_test = test_set.drop(['id'], axis=1)

X_train[cols_fill_0].fillna(0, inplace=True) 
X_test[cols_fill_0].fillna(0, inplace=True) 

for col in cols_fill_mean:
    X_train[col].fillna(train_set[col].mean(), inplace=True)  
    X_test[col].fillna(train_set[col].mean(), inplace=True) 
for col in cols_fill_None:
    X_train[col].fillna('0', inplace=True)  
    X_test[col].fillna('0', inplace=True)  

X_train_drop_oh = X_train.drop(cols_oh, axis=1)
X_test_drop_oh = X_test.drop(cols_oh, axis=1)

X_train_drop_oh.shape, X_test_drop_oh.shape

((196539, 12), (56279, 12))

In [5]:
X_train_drop_oh.head(10)

,时间,小区房屋出租数量,楼层,总楼层,房屋面积,居住状态,卧室数量,厅的数量,卫的数量,出租方式,距离,装修情况
34333,1,0.117188,2,0.581818,0.014068,0,3,1,2,0,0.551202,0.0
57240,2,0.050781,1,0.600000,0.013241,0,2,2,1,0,0.551202,0.0
58959,2,0.121094,1,0.454545,0.017875,0,3,2,2,0,0.551202,0.0
163930,3,0.535156,2,0.563636,0.014399,0,2,2,1,0,0.551202,0.0
171982,3,0.484375,2,0.545455,0.008937,0,1,0,1,0,0.228333,0.0
55511,2,0.257812,1,0.272727,0.012579,0,2,2,1,0,0.551202,0.0
10600,1,0.050781,1,0.600000,0.010096,0,2,2,1,0,0.551202,0.0
5305,1,0.128906,0,0.327273,0.013737,0,3,2,1,0,0.551202,0.0
18314,1,0.054688,1,0.581818,0.013903,0,2,2,1,0,0.551202,0.0
191277,3,0.132812,0,0.527273,0.010013,0,2,1,1,0,0.551202,0.0


In [6]:
from sklearn.preprocessing import LabelEncoder,OneHotEncoder,LabelBinarizer #要先转化为数值，再onehot
def get_oh(cols): #输入需要onehot的list
#     global X_train_oh, X_test_oh
    X_train_oh = np.arange(X_train.shape[0]).reshape(X_train.shape[0],1)
    X_test_oh = np.arange(X_test.shape[0]).reshape(X_test.shape[0],1)
    
    for col in cols:     
        
        col_train = X_train[col].astype('str')
        col_test = X_test[col].astype('str')
        
        oh_fea = LabelBinarizer().fit(col_train) #去onehot特征
        train_oh = oh_fea.transform(col_train)
        test_oh = oh_fea.transform(col_test)        
        print(col,'：',train_oh.shape, test_oh.shape)
        
        X_train_oh = np.append(X_train_oh, train_oh, axis=1)
        X_test_oh = np.append(X_test_oh, test_oh, axis=1)
    
    np.delete(X_train_oh, 0, axis=1) #因为刚初始化时多了一个arange
    np.delete(X_test_oh, 0, axis=1)
    return X_train_oh, X_test_oh

X_train_oh, X_test_oh = get_oh(cols_oh[:])

# 这是先先数值化再onehot的
# direct_train_labelencoded = LabelEncoder().fit_transform(direct_train) #数值化
# print(direct_train_labelencoded, direct_train_labelencoded.shape)
# oh = OneHotEncoder(sparse=False).fit(direct_train_labelencoded.reshape(-1,1)) #要转化为(xxx,1)
# direct_oh = oh.transform(direct_train_labelencoded.reshape(-1,1)) 

# direct_train_oh, direct_test_oh = get_oh('房屋朝向')
# name_train_oh, name_test_oh = get_oh('小区名')

小区名 ： (196539, 5547) (56279, 5547)
房屋朝向 ： (196539, 64) (56279, 64)
区 ： (196539, 15) (56279, 15)
位置 ： (196539, 153) (56279, 153)
地铁线路 ： (196539, 6) (56279, 6)
地铁站点 ： (196539, 119) (56279, 119)


In [7]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler

# 归一化
mms = MinMaxScaler((0,1))
mms.fit(X_train_drop_oh)
X_train_drop_oh = mms.transform(X_train_drop_oh)
X_test_drop_oh = mms.transform(X_test_drop_oh)


#标准化
# ss = StandardScaler()
# ss.fit(X_train_drop_oh)
# X_train_drop_oh = ss.transform(X_train_drop_oh)
# X_test_drop_oh = ss.transform(X_test_drop_oh)


'''
X_train_mms = mms.transform(X_train[cols_nums])
X_test_mms = mms.transform(X_test[cols_nums])

# 替换为归一化之后的属性值
X_train[cols_nums] = X_train_mms[cols_nums]
X_test[cols_nums] = X_tesr_mms[cols_nums]
'''
print()

In [8]:
#小区名太多。。。单独处理
# X_train_oh_name, X_test_oh_name = get_oh([cols_oh[0]]) #因为只有一个值，要加上括号

# from sklearn.decomposition import PCA
# pca = PCA(n_components=1000)
# X_train_oh_name

In [9]:
# dir_index = direct_train.value_counts().index
# list(dir_index)
X_train.shape, X_train_oh.shape, X_test.shape, X_test_oh.shape, 

((196539, 18), (196539, 5905), (56279, 18), (56279, 5905))

In [10]:
#合并两个 X
X_train = np.append(X_train_drop_oh, X_train_oh, axis=1)
X_test = np.append(X_test_drop_oh, X_test_oh, axis=1)
# 5924

#合并多个 
# X_train = np.hstack((X_train, direct_train_oh, name_train_oh)) #横向合并
# X_test = np.hstack((X_test, direct_test_oh, name_test_oh)) #横向合并
X_train.shape, X_test.shape, 

((196539, 5917), (56279, 5917))

In [11]:
# 整体降维
# from sklearn.decomposition import PCA
# pca = PCA(n_components=1000)
# pca.fit(X_train)
# X_train = pca.transform(X_train)
# X_test = pca.transform(X_test)
# X_train.shape, X_test.shape

In [12]:
# 划分数据，调参

from sklearn.model_selection import train_test_split
X_train2, X_test2, y_train2, y_test2 = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

In [13]:
from sklearn.kernel_ridge import KernelRidge
from sklearn import svm
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_squared_error
from sklearn.grid_search import GridSearchCV
from sklearn.grid_search import RandomizedSearchCV
from sklearn.externals import joblib
import time
start = time.time()

# lasso = Lasso(alpha=0.0005,max_iter=10000)
# ridge = Ridge(alpha=60)
# svr = svm.SVR(C=1,kernel='rbf',gamma='auto',epsilon=0.1)
# ker = KernelRidge(alpha=20 ,kernel='polynomial',degree=3 , coef0=0.8)
# ela = ElasticNet(alpha=0.005,l1_ratio=0.08,max_iter=10000)
# bay = BayesianRidge()
# xgb = XGBRegressor()
lsvr = svm.LinearSVR(C=1)
rf = RandomForestRegressor(n_estimators=100, max_features=0.6, max_depth=200,
                           min_samples_split=2, verbose=1, n_jobs=-1,) #服务器10:122s; 100:665s 200:957s
# n为20时:0.39/2.23；40时0.34,2.15；50时0.35/2.19。特征0.99和n20的数据差不多，0.8时0.4/2.27
xgb = XGBRegressor(n_estimators=15, reg_alpha=0, reg_lambda=1, learning_rate=0.2, 
                   min_child_weight=1, max_depth=40, silent=False, n_jobs=40)  

reg = rf
reg.fit(X_train2, y_train2, )
# joblib.dump(reg,'try_mean_rf200_f055.pkl')


# param_svr = {'kernel':['rbf', 'poly','linear'],
#              'C':scipy.stats.expon(scale=1), #指数分布，mean≈std≈1             
#             }

# param_rf = {#'n_estimators':[20,50,100], #100>50>20>10
# #         'max_features':[0.6, 0.55],#0.5>0.45&0.55>0.5       0.6>0.7>0.8>0.9>auto≈0.99;0.7>sqrt&log2*0.5
#         'max_depth':[100, None], #None>100≈70>50>20>10
# #         'min_samples_split':[2,5] #2>5>10>20>50
# #         'min_samples_leaf':[1,10] #1>10
#        }  
# param_xgb = {
#         'n_estimators':[50,100], 
#         'max_depth':[15, 20], 
# #         'learning_rate':[0.1,0.2], #0.2>0.15&0.3
# #         'reg_lambda':[1, 1.2]
#        } 
# gridsearch = GridSearchCV(reg, param_grid=param_rf, scoring='neg_mean_squared_error', iid=False, cv=4, n_jobs=-1)  
# gridsearch.fit(X_train,y_train)  
# print(gridsearch.grid_scores_,'\n',gridsearch.best_params_, gridsearch.best_score_, gridsearch.best_estimator_)

# randsearch = RandomizedSearchCV(reg, param_distributions=param_svr, scoring='neg_mean_squared_error', cv=3, n_iter=10, n_jobs=-1)
# randsearch.fit(X_train,y_train)  
# print(randsearch.best_params_, randsearch.best_score_)

# 交叉验证
# score = cross_val_score(reg, X_train2, y_train2, cv=3,  n_jobs=-1)
# print(score)

# reg = joblib.load('model.pkl')
mse_train2 = mean_squared_error(y_train2, reg.predict(X_train2))
mse_test2 = mean_squared_error(y_test2, reg.predict(X_test2))
print(mse_train2, mse_test2)
print(reg)
print('时间：',time.time() - start)

/home/amax/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/home/amax/anaconda3/lib/python3.6/site-packages/sklearn/grid_search.py:42: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed: 13.7min finished
[Parallel(n_jobs=40)]: Done 100 out of 100 | elapsed:    2.4s finished


0.2882434505897285 2.374907872453527
RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=200,
           max_features=0.6, max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=-1,
           oob_score=False, random_state=None, verbose=1, warm_start=False)
时间： 842.9919757843018


[Parallel(n_jobs=40)]: Done 100 out of 100 | elapsed:    0.9s finished


In [14]:
# 用最优参数fit

# reg = RandomForestRegressor(n_estimators=50, max_depth=15, min_samples_split=50, n_jobs=-1)
# reg.fit(X_train2, y_train2, )
# mse_train2 = mean_squared_error(y_train2, reg.predict(X_train2))
# mse_test2 = mean_squared_error(y_test2, reg.predict(X_test2))
# print(mse_train2, mse_test2)

In [15]:
y_pred = reg.predict(X_test)
y_pred.shape

[Parallel(n_jobs=40)]: Done 100 out of 100 | elapsed:    1.1s finished


(56279,)

In [16]:
# with open('output.csv', 'w') as f:
#     id = test_set['id']
#     f.write('id,price'+'\n')
#     for i in range(len(y_pred)):
#         f.write(id[i]+','+str(y_pred[i])+'\n')

# keras

In [17]:
# from keras import models
# from keras import layers

# network = models.Sequential()
# network.add(layers.Dense(128, activation='relu', input_shape=(X_train2.shape[1], )))
# # network.add(layers.Dense(128, activation='relu'))
# network.add(layers.Dense(64, activation='relu'))
# network.add(layers.Dense(1))
# network.compile(optimizer='rmsprop',
#                loss='mse',
#                metrics=['mse'])

# network.fit(X_train2, y_train2, batch_size=64, epochs=10)

# train_mse =network.evaluate(X_train2, y_train2)
# test_mse =network.evaluate(X_test2, y_test2)
# print(train_mse, test_mse)